In [1]:
'''If gives out error, ignore'''
from pyspark import SparkContext
sc=SparkContext()

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [95]:
TWEET_PATH = 'D:/Datahack/bigdatathon-2017/data/'
TWEET_FILE = 'tweetssentiment.txt'
TWEET_PATHFILE = TWEET_PATH+TWEET_FILE
ITEM_PATH = 'D:/Datahack/bigdatathon-2017/scrapers/farfetch/'
ITEM_FILE = 'kv-data-hashed-forvictor'
ITEM_PATHFILE = ITEM_PATH+ITEM_FILE



Tweets = (sc.textFile(TWEET_PATHFILE, 4).distinct().map(lambda x : x.split(',')).cache())


In [96]:
Items = (sc.textFile(ITEM_PATHFILE, 4).map(lambda x : x.split('","')).cache())

In [97]:
stopwords = set(["a","b", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along",
                  "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone",
                  "anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming",
                  "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom",
                  "but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
                  "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every",
                  "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly",
                  "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her",
                  "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if",
                  "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many",
                  "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself",
                  "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing",
                  "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
                  "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem",
                  "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
                  "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the",
                  "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv",
                  "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward",
                  "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were",
                  "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while",
                  "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you",
                  "your", "yours", "yourself", "yourselves", "the","add","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]);


In [98]:
import re
split_regex = r'[a-zA-Z0-9_]+'
def simpleTokenize(string):
    """ A simple implementation of input string tokenization
    Args:
        string (str): input string
    Returns:
        list: a list of tokens
    """
    string = re.sub(r'([^\s\w(?<_)])+', ' ',string.lower())
    return re.findall(split_regex, string)


def removeQuotes(s):
    """ Remove quotation marks from an input string
    Args:
        s (str): input string that might have the quote "" characters
    Returns:
        str: a string without the quote characters
    """
    return ''.join(i for i in s if i!='"')

def tokenize(string):
    """ An implementation of input string tokenization that excludes stopwords
    Args:
        string (str): input string
    Returns:
        list: a list of tokens without stopwords
    """
    string = removeQuotes(string)
    def remove_if_not_substring(l1, l2):
        return [i for i in l1 if not any(j == i for j in list(l2))]
    string = remove_if_not_substring(simpleTokenize(string), stopwords)
    return string



TweetsPARSED = Tweets.map(lambda x: (x[1],(x[0],tokenize(x[2]),x[3])))
ItemsPARSED = Items.map(lambda x: (removeQuotes(x[9]),(removeQuotes(x[2]),re.sub('\s+','_', x[3]),removeQuotes(x[4]),tokenize(x[5]))))
##TweetsPARSED = Tweets.map(lambda x: (removeQuotes(x[1]),1))
##ItemsPARSED = Items.map(lambda x: (removeQuotes(x[9]),1))



In [107]:
JoinedRDD = TweetsPARSED.join(ItemsPARSED).map(lambda x: (x[0],(x[1][0]+x[1][1])))

In [112]:
COM_ATTRI_ITEMS = JoinedRDD.map(lambda x: (x[0],set(x[1][1]+x[1][6])))
COM_ATTRI_ITEMS.combineByKey(str,lambda a,b:a+b,lambda a,b:a+b).take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 141.0 failed 1 times, most recent failure: Lost task 1.0 in stage 141.0 (TID 349, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 177, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 172, in process
  File "c:\users\victor\appdata\local\programs\python\python36\lib\site-packages\pyspark\rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\users\victor\appdata\local\programs\python\python36\lib\site-packages\pyspark\rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\users\victor\appdata\local\programs\python\python36\lib\site-packages\pyspark\rdd.py", line 346, in func


In [51]:
from pyspark.sql.types import Row

col_names_tweets = ['ItemID', 'UID', 'Tweet', 'sent_score']
col_names_items = ['ItemID', 'ItemNAME', 'ItemDESIGNER', 'ItemPRICE', 'ItemDESC']
def f(x,col_names):
    d = {}
    for i in range(len(x)):
        d[col_names[i]] = x[i]
    return d

TweetsDF = TweetsPARSED.map(lambda x: Row(**f(x,col_names_tweets))).toDF()
ItemsDF = ItemsPARSED.map(lambda x: Row(**f(x,col_names_items))).toDF()